In [1]:
import torch
import pandas as pd
from torch import nn
from torchvision import transforms

from functools import partial

from collections import defaultdict



import re, math, sys
import numpy as np
import random

import os
from tqdm import tqdm

from PIL import Image
import importlib

from matplotlib import pyplot as plt

### Load data

In [2]:
stimuli_df = None
import utils
importlib.reload(utils)
stimuli_df = utils.STIMULI_DF

# add an extra "../" tot he data paths to create ../../VALSE_data
stimuli_df['img_path'] = [os.path.join("..",path) for path in stimuli_df['img_path']]

stimuli_df.head(2)


# import entire VALSE but only relevant linguistic phenomena
all_valse_df = utils.ENTIRE_DATASET_DF
all_valse_df = all_valse_df[all_valse_df['linguistic_phenomena'].isin(['actions','relations','existence'])]

# create 'img_path' column which has correct path to all the img files of valse dataset
all_valse_df['img_path'] = [os.path.join("../../VALSE_data/",path) for path in all_valse_df['local_img_path']]
all_valse_df.drop(['local_img_path'], axis=1)
all_valse_df.head(2)



/var/folders/3l/2lt99jp97mvbvsj3_p5hh8f80000gn/T/ipykernel_90032/1684243812.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_valse_df['img_path'] = [os.path.join("../../VALSE_data/",path) for path in all_valse_df['local_img_path']]


,dataset,dataset_idx,image_file,linguistic_phenomena,caption,answer,classes,split,original_split,classes_foil,...,key,declarative_statement,declarative_statement_foils,provenance_of_foils,foiling_method,orig_caption,only_caption,local_img_path,include_in_experiment,img_path
1000,coco2017,212732,000000462576.jpg,relations,Breakfast items including juice are on the table.,NaN,on,NaN,val,off,...,relations:coco_2017:212732,NaN,NaN,NaN,NaN,NaN,NaN,images/coco2017/000000462576.jpg,True,../../VALSE_data/images/coco2017/000000462576.jpg
1001,coco2017,528685,000000294162.jpg,relations,The white laptop is turned off but left open.,NaN,off,NaN,val,on,...,relations:coco_2017:528685,NaN,NaN,NaN,NaN,NaN,NaN,images/coco2017/000000294162.jpg,True,../../VALSE_data/images/coco2017/000000294162.jpg


# Setting up FLAVA

### Load model

In [3]:
global model
import requests
from transformers import FlavaProcessor, FlavaForPreTraining

flava_model = FlavaForPreTraining.from_pretrained("facebook/flava-full").eval().to("cpu")
flava_processor = FlavaProcessor.from_pretrained("facebook/flava-full")

model_name = "FLAVA"

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


In [4]:
flava_processor

FlavaProcessor:
- image_processor: FlavaImageProcessor {
  "codebook_crop_size": {
    "height": 112,
    "width": 112
  },
  "codebook_do_center_crop": true,
  "codebook_do_map_pixels": true,
  "codebook_do_normalize": true,
  "codebook_do_rescale": true,
  "codebook_do_resize": true,
  "codebook_image_mean": [
    0.0,
    0.0,
    0.0
  ],
  "codebook_image_std": [
    1.0,
    1.0,
    1.0
  ],
  "codebook_resample": 1,
  "codebook_rescale_factor": 0.00392156862745098,
  "codebook_size": {
    "height": 112,
    "width": 112
  },
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "FlavaFeatureExtractor",
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "FlavaImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "input_size_patches": 14,
  "mask_group_max_aspect_ratio": null,
  

# Test model

Model card with helpful instructions: https://huggingface.co/facebook/flava-full

Caption: There is at least one bike on the rack.
Foil: There are no bikes on the rack.
Caption: There is at least one player shown.
Foil: There are no players shown.


In [9]:
row = stimuli_df.iloc[7]
img_path1, caption1, foil1 = row['img_path'], row['caption'], row['foil']
image1 = Image.open(img_path1)

row = stimuli_df.iloc[8]
img_path2, caption2, foil2 = row['img_path'], row['caption'], row['foil']
image2 = Image.open(img_path2)

plt.imshow(image1)
print(f"Caption: {caption1}\nFoil: {foil1}")


plt.imshow(image2)
print(f"Caption: {caption2}\nFoil: {foil2}")

**Make sure it can take multiple images at once**

In [10]:

images = [image1, image1, image2, image2]
text = [caption1, foil1, caption1, foil1] # we repeat the same caption and foil across each

with torch.no_grad():

    # encode text and image input together for caption and foil for potentially multiple images
    input = flava_processor(text=text,
                            images=[image.convert('RGB') for image in images], 
                            return_tensors="pt", 
                            max_length=77, padding=True, 
                            return_codebook_pixels=True, 
                            return_image_mask=True).to("cpu")

    output = flava_model(**input)



    # similarity matrix has size (2 x nr of images, 2 x nr_of_images). 
    # Each row is an image as image1, image1, image2, image2, etc, 
    # Each column is a caption like caption1, foil1, caption2, foil2, etc.
    similarity_matrix = output.contrastive_logits_per_image # output of the format [ [img1_caption_score, img1_foil_score],   
                                                                # [img2_caption_score, img2_foil_score]  ]
    
print(similarity_matrix)
# we only want the DIAGONAL values -- these align the right image to the right captions and foils
scores_by_image = []

for i in range(int(len(images)/2)):
    caption_score = similarity_matrix[2*i, 2*i]
    foil_score = similarity_matrix[2*i+1, 2*i+1]
    scores_by_image.append([caption_score, foil_score])

scores_by_image = np.array(scores_by_image)

scores_by_image


 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tensor([[17.9686, 17.3375, 17.9686, 17.3375],
        [17.9686, 17.3375, 17.9686, 17.3375],
        [ 3.6838,  3.9386,  3.6838,  3.9386],
        [ 3.6838,  3.9386,  3.6838,  3.9386]])


array([[17.968575 , 17.337477 ],
       [ 3.6837916,  3.938579 ]], dtype=float32)

**How to interpret the similarity matrix:**

Each row represents an image. Each column represents a caption. The dot products are the similarity scores.

So if we have lots of images x text, we only really care about the diagonal. Say we have 3 images and a caption, foil for each.

Then output[0,0] is the caption similarity for the first image, output[1,1] is the foil similarity for the frist image, output[2,2] is the caption similarity for the first image, etc.

# Run on data

### Define model prediction function (for SHAP -- taking list of images and caption/foil as input)

In [4]:
text_out = []


def model_predict(images, caption, foil, single_image = False, single_score = True, verbose = False):
    """Needs CAPTION and FOIL to be defined outside the function,
    caption and foil kept constant within the function for each image

    images needs to be a LIST, even if only one member


    single score = True for SHAP! (this means one number returned for each image)
    single_image = False for SHAP! (single_image means only one value or set of values is returned, assuming there is 1 image only)
    """

    global flava_model


    # build speecial type of caption and images list required by flava processor (dims have to be equal for whatever reason)
    caption_foil_list = []
    images_list = []
    for image in images:
        caption_foil_list += [caption, foil]  # should be [caption, foil, caption, foil] repeating len(images) times!
        images_list += [image, image]   # should be [image1, image1, image2, image2, ...] to end of images


    with torch.no_grad():

        # encode text and image input together for caption and foil for potentially multiple images
        input = flava_processor(text=caption_foil_list,
                                images=images_list, 
                                return_tensors="pt", 
                                max_length=77, padding=True, 
                                return_codebook_pixels=True, 
                                return_image_mask=True).to("cpu")

        output = flava_model(**input)



        # similarity matrix has size (2 x nr of images, 2 x nr_of_images). 
        # Each row is an image as image1, image1, image2, image2, etc, 
        # Each column is a caption like caption, foil, caption, foil, etc.
        similarity_matrix = output.contrastive_logits_per_image # output of the format [ [img1_caption_score, img1_foil_score],   
                                                                    # [img2_caption_score, img2_foil_score]  ]
        
    if verbose: print("Similarity matrix: (row is image1, image1, img2, img2) (column is caption foil caption foil ...)", similarity_matrix)
    # we only want the DIAGONAL values -- these align the right image to the right captions and foils
    scores_by_image = []

    # the scores are all on the DIAGONAL of the similarity matrix
    for i in range(int(len(images))):
        caption_score = similarity_matrix[2*i, 2*i]
        foil_score = similarity_matrix[2*i+1, 2*i+1]
        scores_by_image.append([caption_score, foil_score])


    # finally generate scores list of this format [ [img1_caption, img2_foil], [img2_caption, img2_foil],...  ]
    scores = np.array(scores_by_image)

    # if a single score, return a single score for each image (useful for SHAP)
    if single_score:
        scores = scores[:, 0] - scores[:, 1]  

    # if single image, return the first element of the list
    if single_image:
        return scores[0]
    else: # otherwise return a list
        return scores

# test on above example
print("Score for each caption-image pair for two images:", model_predict([image1, image2], caption1, foil1, single_image = False, single_score = False, verbose=True))
print("Caption foil score DIFFERENCE for two images:", model_predict([image1, image2], caption1, foil1, single_image = False, single_score = True))



NameError: name 'image1' is not defined

### Define model output caption for dataframes (taking dataframe as input)

Difference: the SHAP prediction function works with a list of images for a single set of captions. 

This one works with a dataframe where each SINGLE image is paired with one caption/foil and an output generated.

In [16]:
def model_output_dataframe(df):
    
    """Given a dataframe of images, captions, and foils, returns a dataframe of model outputs, one for each row in the dataframe"""

    results = defaultdict(list)

    for _, row in tqdm(df.iterrows()):

        # copy over those things that should be copied directly
        for name in ["img_path", "caption", "foil", "linguistic_phenomena"]:
            results[name].append(row[name])

        # prepare data point as inputs to test model on
        image = Image.open(row["img_path"]).convert('RGB')

        caption, foil = row["caption"], row["foil"]
        
        
        # create model input with BOTH captions
        try:
            model_results = model_predict([image], caption, foil, single_image = True, single_score = False)
        except Exception as e:
            print(f"Trouble processing {row['img_path']}")
            print(e)
            continue
        
        caption_score, foil_score = model_results[0].item(), model_results[1].item()
        caption_foil_diff = caption_score - foil_score
        
        # save relevant fields to the dictionary to either {caption} or {foil} (based on which it is)
        results["model_pred_caption"].append(caption_score) # logit
        results["model_pred_foil"].append(foil_score)
        results["model_pred_diff"].append(caption_foil_diff)

    print(results)

    
    model_output = pd.DataFrame.from_dict(results)


    return model_output

### Generate model output for each data point in STIMULI dataframe (without maskng)

0it [00:00, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
1it [00:00,  1.23it/s]`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you ar

defaultdict(<class 'list'>, {'img_path': ['../../VALSE_data/images/v7w/v7w_2318073.jpg', '../../VALSE_data/images/v7w/v7w_2323857.jpg', '../../VALSE_data/images/v7w/v7w_2336798.jpg', '../../VALSE_data/images/v7w/v7w_2349587.jpg', '../../VALSE_data/images/v7w/v7w_2352298.jpg', '../../VALSE_data/images/v7w/v7w_2360434.jpg', '../../VALSE_data/images/v7w/v7w_2362476.jpg', '../../VALSE_data/images/v7w/v7w_2389519.jpg', '../../VALSE_data/images/v7w/v7w_2393355.jpg', '../../VALSE_data/images/v7w/v7w_2411942.jpg', '../../VALSE_data/images/v7w/v7w_2417412.jpg', '../../VALSE_data/images/swig/speaking_84.jpg', '../../VALSE_data/images/swig/tripping_125.jpg', '../../VALSE_data/images/swig/punching_28.jpg', '../../VALSE_data/images/swig/handcuffing_40.jpg', '../../VALSE_data/images/swig/caressing_51.jpg', '../../VALSE_data/images/swig/urinating_251.jpg', '../../VALSE_data/images/swig/foraging_17.jpg', '../../VALSE_data/images/swig/communicating_34.jpg', '../../VALSE_data/images/swig/complaining_6.j

In [17]:

stimuli_output_df = model_output_dataframe(stimuli_df)

# export results
stimuli_output_df.to_pickle(f"xai_output/{model_name}_stimuli_output_scores.pickle")

# graph outcome
stimuli_output_df.boxplot(column='model_pred_diff', by='linguistic_phenomena')
plt.title(f"{model_name} pred difference for experiment stimuli")
plt.show()


### Generate model output for ENTIRE VALSE DATASET for 3 linguistic phenomena

Assuming  only for 3 phenomena

0it [00:00, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
1it [00:00,  1.28it/s]`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you ar

defaultdict(<class 'list'>, {'img_path': ['../../VALSE_data/images/coco2017/000000462576.jpg', '../../VALSE_data/images/coco2017/000000294162.jpg', '../../VALSE_data/images/coco2017/000000144333.jpg', '../../VALSE_data/images/coco2017/000000181666.jpg', '../../VALSE_data/images/coco2017/000000482436.jpg', '../../VALSE_data/images/coco2017/000000153299.jpg', '../../VALSE_data/images/coco2017/000000337055.jpg', '../../VALSE_data/images/coco2017/000000526728.jpg', '../../VALSE_data/images/coco2017/000000402519.jpg', '../../VALSE_data/images/coco2017/000000490413.jpg', '../../VALSE_data/images/coco2017/000000178618.jpg', '../../VALSE_data/images/coco2017/000000140286.jpg', '../../VALSE_data/images/coco2017/000000033368.jpg', '../../VALSE_data/images/coco2017/000000034760.jpg', '../../VALSE_data/images/coco2017/000000320554.jpg', '../../VALSE_data/images/coco2017/000000141671.jpg', '../../VALSE_data/images/coco2017/000000068628.jpg', '../../VALSE_data/images/coco2017/000000003255.jpg', '../

In [18]:
RUN_ON_WHOLE_DATASET = True # can be turned off when we don't need this

if RUN_ON_WHOLE_DATASET:

    valse_output_df = model_output_dataframe(all_valse_df)

    # export results
    valse_output_df.to_pickle(f"xai_output/{model_name}_all_of_valse_output_scores.pickle")

    # graph outcome
    valse_output_df.boxplot(column='model_pred_diff', by='linguistic_phenomena')
    plt.title(f"{model_name} pred difference for all of VALSE")
    plt.show()

## Build SHAP pipeline

### Masking functions

In [ ]:
from shap_functions import vector_to_masked_images, blur_image


### Test masking

In [ ]:
# image = Image.open("../../VALSE_data/images/coco2017/000000122672.jpg")
# image = np.array(image.resize((224,224)))

# blurred_image = blur_image(image)

# test_mask_array = np.array([
#     [1, 0, 0, 1],
#     [0, 1, 1, 0],
#     [1, 0, 0, 0],
#     [0, 0, 1, 0]
# ])

# masked_img = vector_to_masked_images([test_mask_array], image, blurred_image, N_ROWS=4)[0]
# plt.imshow(masked_img)

### SHAP function

In [5]:
import shap_functions
importlib.reload(shap_functions)

<module 'shap_functions' from '/Users/eduardsaakashvili/Desktop/UU/Thesis/code/es-thesis-repo/data/xai_maps/shap_functions.py'>

# Generate and export SHAP maps

### Generate maps (in smaller batchs)

In [ ]:
import pandas as pd
from collections import defaultdict
import os

VERBOSE = 1 
DISPLAY_PLOTS = False
EXPORT_PLOTS = True
CHUNK_SIZE = 11  # nr of rows to process at a time
START_INDEX = 0  # default start index is 0

output = defaultdict(list)

# find total number of chunks needed
total_rows = len(stimuli_df)
num_chunks = (total_rows - START_INDEX) // CHUNK_SIZE + ((total_rows - START_INDEX) % CHUNK_SIZE > 0)

all_dfs = []  # list to store each chunk's DF

# loop through DF one chunk at a time
for chunk_start in range(START_INDEX, total_rows, CHUNK_SIZE):
    chunk_end = min(chunk_start + CHUNK_SIZE, total_rows)
    print(f"Processing rows {chunk_start} to {chunk_end} out of {total_rows}")

    # chunk output is a separate list
    chunk_output = defaultdict(list)

    # iterate through rows in chunk
    for index, row in stimuli_df.iloc[chunk_start:chunk_end].iterrows():
        print(f"Processing stimulus {index + 1} out of {total_rows}")

        # Copy main characteristics
        for name in ["img_path", "caption", "foil", "linguistic_phenomena"]:
            chunk_output[name].append(row[name])

        # Calculate shap map
        try:
            shap_map, n_variants = shap_functions.generate_shap(row['img_path'], row['caption'], row['foil'],
                                                                prediction_function=model_predict,
                                                                model=flava_model,
                                                                model_name=model_name,
                                                                verbose=VERBOSE,
                                                                plots=DISPLAY_PLOTS,
                                                                export_plots=EXPORT_PLOTS,
                                                                return_variant_count=True)
            chunk_output["model_shap_map"].append(shap_map)
            chunk_output['n_image_variants'].append(n_variants)

        except Exception as e:
            chunk_output['model_shap_map'].append(f"Error: {str(e)}")
            chunk_output['n_image_variants'].append("Error")

    # convert the chunk output to a DF and save it
    chunk_df = pd.DataFrame.from_dict(chunk_output)
    chunk_df.to_pickle(f"xai_output/{model_name}_shap_maps_chunk_{chunk_start//CHUNK_SIZE}.pickle")



In [8]:
# merge all chunk outputs into one

# open aeach chunk DF, into a list
all_dfs = [pd.read_pickle(os.path.join("xai_output", filename)) for filename in os.listdir("xai_output") if "chunk" in filename and model_name in filename]

# merge all the lists
final_model_shap_df = pd.concat(all_dfs, ignore_index=True)
final_model_shap_df.to_pickle(f"xai_output/{model_name}_shap_maps.pickle")

In [7]:
# for key in output.keys():
#     print(key, len(output[key]))

### Export resulting dataframe

In [23]:
final_model_shap_df.to_pickle(f"xai_output/{model_name}_shap_maps.pickle")

### Find which if any stimuli threw an error

In [24]:
final_model_shap_df[final_model_shap_df['model_shap_map'].apply(isinstance, args=(str,))]

,img_path,caption,foil,linguistic_phenomena,model_shap_map,n_image_variants


In [ ]:
plt.imshow(final_clip_shap_df['clip_shap_map'][0])

NameError: name 'final_clip_shap_df' is not defined